<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span><ul class="toc-item"><li><span><a href="#Тестирование" data-toc-modified-id="Тестирование-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Тестирование</a></span></li></ul></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

В данном проекте обучим модель, которая классифицирует токсичные комментарии.

## Подготовка

Импорт библиотек:

In [1]:
import pandas as pd
import numpy as np
import nltk
import re

from nltk.corpus import stopwords as nltk_stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from lightgbm import LGBMClassifier
from nltk.corpus import wordnet

SEED = 42

In [2]:
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/lefantino/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/lefantino/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/lefantino/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to /Users/lefantino/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
df = pd.read_csv('toxic_comments.csv')

In [4]:
df = df.drop('Unnamed: 0', axis=1)

In [5]:
df.head(5)

text  toxic
0  Explanation\nWhy the edits made under my usern...      0
1  D'aww! He matches this background colour I'm s...      0
2  Hey man, I'm really not trying to edit war. It...      0
3  "\nMore\nI can't make any real suggestions on ...      0
4  You, sir, are my hero. Any chance you remember...      0

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159292 non-null  object
 1   toxic   159292 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


In [7]:
df.duplicated().sum()

0

In [8]:
len(max(df['text'], key=len))

5000

In [9]:
len(min(df['text'], key=len))

5

In [10]:
df['toxic'].value_counts(normalize=True)

0    0.898388
1    0.101612
Name: toxic, dtype: float64

В датасете 2 признака:
* text - комментарий от пользователя на английском языке, самый длинный комментарий 5000 символов, самый короткий - 5 символов.
* target - целевая переменная, метка 0 или 1, характеризующая комментарий как токсичный

В самом датасете 159292 строки, нет пропусков, нет дубликатов.  
Изучим, что из себя представляют комментарии:

In [11]:
df.loc[0, 'text']

"Explanation\nWhy the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired now.89.205.38.27"

In [12]:
df.loc[1, 'text']

"D'aww! He matches this background colour I'm seemingly stuck with. Thanks.  (talk) 21:51, January 11, 2016 (UTC)"

In [13]:
df.loc[3000, 'text']

"Ringmail\nI don't know if he regards himself as an expert, I've sweated enough to know for sure that a broigne is not a victorian mistake, and to find citable sources for it. I understand that the English language wor;ld lacks proper history on the subject, but from a continental point of view, there's no mistake possible that ringmail is different from chainmail and was used in the period between the fall of Rome and the comeback of chainmail in the late 1100s. Also, the high handed way in which he remade all the article with no regard for the effots of those who came before him, and without even setting up an account for people to discuss the matter with him wxas, to my view, qualification for vandalism.\n\nI'm quite willing to include the bit about representations making it difficult to differenciate between chainmail and ringmail, and the possibility of misconceptions in the 1800s, but the basic matter as established with my collaboration stands, even if it has to be edited furthe

Сами комментарии представляют из себя строку, состоящую из предложений или законченных фраз, встречаются так же лишние символы в строках, сокращения, даты, цифры.

Разобъем комментарии на токены и лемматизируем их:

In [14]:
w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
lemmatizer = nltk.stem.WordNetLemmatizer()

def get_wordnet_pos(word):
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)

def lemmatize_text(text):
    return ' '.join([lemmatizer.lemmatize(w, get_wordnet_pos(w)) for w in nltk.word_tokenize(text)])

In [15]:
%%time
df['lemm_text'] = df.text.apply(lemmatize_text)

CPU times: user 12min 45s, sys: 43.7 s, total: 13min 29s
Wall time: 13min 32s


Что получилось:

In [16]:
df['lemm_text'].head()

0    Explanation Why the edits make under my userna...
1    D'aww ! He match this background colour I 'm s...
2    Hey man , I 'm really not try to edit war . It...
3    `` More I ca n't make any real suggestion on i...
4    You , sir , be my hero . Any chance you rememb...
Name: lemm_text, dtype: object

Так же необходимо почистить текст от лишних символов, оставив только слова:

In [17]:
replacement = {
    "'m":" am",
    "'v":" have",
    "'re":" are",
    "'ll":" will",
    "'t":" not",
}
for k,v in replacement.items():
    df['lemm_text'] = df['lemm_text'].str.replace(k, v)

В идеальном случае необходимо намного тщательнее обработать текст, заменить различные абревиатуры или сленговые сокращения и т.д. Так как проект учебный, не будем заострять на этом внимание

In [18]:
def cleaning_lemm_text(text):
    text = text.lower()
    text = re.sub(r'[^a-zA-Z ]', ' ', text)
    text = text.strip()
    return ' '.join(text.split())

In [19]:
df['lemm_text'] = df['lemm_text'].apply(cleaning_lemm_text)

In [20]:
df['lemm_text'].head()

0    explanation why the edits make under my userna...
1    d aww he match this background colour i am see...
2    hey man i am really not try to edit war it s j...
3    more i ca n not make any real suggestion on im...
4    you sir be my hero any chance you remember wha...
Name: lemm_text, dtype: object

In [21]:
df['lemm_text'][0]

'explanation why the edits make under my username hardcore metallica fan be revert they be n not vandalism just closure on some gas after i vote at new york dolls fac and please do n not remove the template from the talk page since i am retire now'

Предобработка текста завершена, разделим данные на обучающую и тестовые выборки. Целевая переменная - toxic, признак - лемматизированный текст.

In [22]:
features = df['lemm_text']
target = df['toxic']

In [23]:
X_train, X_test, y_train, y_test = train_test_split(features, target, 
                                                    test_size=0.25, stratify=target, random_state=SEED)

In [24]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((119469,), (119469,), (39823,), (39823,))

In [25]:
y_train.value_counts(normalize=True)

0    0.898384
1    0.101616
Name: toxic, dtype: float64

In [26]:
y_test.value_counts(normalize=True)

0    0.8984
1    0.1016
Name: toxic, dtype: float64

## Обучение

In [27]:
stopwords = set(nltk_stopwords.words('english'))

In [28]:
cv = StratifiedKFold(n_splits=3)

In [29]:
model_LR = LogisticRegression(random_state=SEED, penalty='l1', solver='liblinear')
model_LGBM = LGBMClassifier(random_state=SEED)

In [30]:
%%time
pipe = Pipeline([
    ('tfidf', TfidfVectorizer(stop_words=stopwords)),
    ('clf', model_LR)
])

params = {
    'clf__C':np.logspace(-3,3,20)
}

grid_LR = GridSearchCV(pipe, param_grid = params, scoring='f1', cv=cv)
grid_LR.fit(X_train, y_train)

CPU times: user 4min 40s, sys: 2.76 s, total: 4min 42s
Wall time: 4min 43s


GridSearchCV(cv=StratifiedKFold(n_splits=3, random_state=None, shuffle=False),
             estimator=Pipeline(steps=[('tfidf',
                                        TfidfVectorizer(stop_words={'a',
                                                                    'about',
                                                                    'above',
                                                                    'after',
                                                                    'again',
                                                                    'against',
                                                                    'ain',
                                                                    'all', 'am',
                                                                    'an', 'and',
                                                                    'any',
                                                                    'are',
                                                                    'aren',
                                                                    "aren't",
                                                                    'as', 'at',
                                                                    'be',
                                                                    'because',
                                                                    'been',
                                                                    'before',
                                                                    'being',
                                                                    'below',
                                                                    'between',
                                                                    'both',
                                                                    'but', 'by',
                                                                    'can',
                                                                    'couldn',
                                                                    "couldn't...
                                                           solver='liblinear'))]),
             param_grid={'clf__C': array([1.00000000e-03, 2.06913808e-03, 4.28133240e-03, 8.85866790e-03,
       1.83298071e-02, 3.79269019e-02, 7.84759970e-02, 1.62377674e-01,
       3.35981829e-01, 6.95192796e-01, 1.43844989e+00, 2.97635144e+00,
       6.15848211e+00, 1.27427499e+01, 2.63665090e+01, 5.45559478e+01,
       1.12883789e+02, 2.33572147e+02, 4.83293024e+02, 1.00000000e+03])},
             scoring='f1')

In [31]:
print(grid_LR.best_estimator_)
print(grid_LR.best_score_)

Pipeline(steps=[('tfidf',
                 TfidfVectorizer(stop_words={'a', 'about', 'above', 'after',
                                             'again', 'against', 'ain', 'all',
                                             'am', 'an', 'and', 'any', 'are',
                                             'aren', "aren't", 'as', 'at', 'be',
                                             'because', 'been', 'before',
                                             'being', 'below', 'between',
                                             'both', 'but', 'by', 'can',
                                             'couldn', "couldn't", ...})),
                ('clf',
                 LogisticRegression(C=2.976351441631316, penalty='l1',
                                    random_state=42, solver='liblinear'))])
0.7795868701397487


In [32]:
%%time
pipe = Pipeline([
    ('tfidf', TfidfVectorizer(stop_words=stopwords)),
    ('clf', model_LGBM)
])

params = {
    'clf__n_estimators':[100,500],
    'clf__learning_rate':[0.01, 0.1, 0.25, 0.5]
}

grid_LGBM = GridSearchCV(pipe, param_grid = params, scoring='f1', cv=cv)
grid_LGBM.fit(X_train, y_train)

CPU times: user 58min 6s, sys: 5min 24s, total: 1h 3min 30s
Wall time: 10min 51s


GridSearchCV(cv=StratifiedKFold(n_splits=3, random_state=None, shuffle=False),
             estimator=Pipeline(steps=[('tfidf',
                                        TfidfVectorizer(stop_words={'a',
                                                                    'about',
                                                                    'above',
                                                                    'after',
                                                                    'again',
                                                                    'against',
                                                                    'ain',
                                                                    'all', 'am',
                                                                    'an', 'and',
                                                                    'any',
                                                                    'are',
                                                                    'aren',
                                                                    "aren't",
                                                                    'as', 'at',
                                                                    'be',
                                                                    'because',
                                                                    'been',
                                                                    'before',
                                                                    'being',
                                                                    'below',
                                                                    'between',
                                                                    'both',
                                                                    'but', 'by',
                                                                    'can',
                                                                    'couldn',
                                                                    "couldn't", ...})),
                                       ('clf',
                                        LGBMClassifier(random_state=42))]),
             param_grid={'clf__learning_rate': [0.01, 0.1, 0.25, 0.5],
                         'clf__n_estimators': [100, 500]},
             scoring='f1')

In [33]:
print(grid_LGBM.best_estimator_)
print(grid_LGBM.best_score_)

Pipeline(steps=[('tfidf',
                 TfidfVectorizer(stop_words={'a', 'about', 'above', 'after',
                                             'again', 'against', 'ain', 'all',
                                             'am', 'an', 'and', 'any', 'are',
                                             'aren', "aren't", 'as', 'at', 'be',
                                             'because', 'been', 'before',
                                             'being', 'below', 'between',
                                             'both', 'but', 'by', 'can',
                                             'couldn', "couldn't", ...})),
                ('clf', LGBMClassifier(n_estimators=500, random_state=42))])
0.77162729415369


In [34]:
columns = ['LogisticRegression', 'LGBM']
scores = [grid_LR.best_score_, grid_LGBM.best_score_]
results = pd.DataFrame(index=['cv'], columns=columns)
results.loc['cv'] = scores

In [35]:
results

LogisticRegression      LGBM
cv           0.779587  0.771627

Обе модели на кросс-валидационных выборках смогли превысить требуемое значение качества, протестируем их на отложенной выборке

In [36]:
models = [grid_LR.best_estimator_, grid_LGBM.best_estimator_]

### Тестирование

Протестируем модели на отложенной выборке:

In [37]:
scores_test = [f1_score(y_test, x.predict(X_test)) for x in models]

In [38]:
results.loc['test'] = scores_test

In [39]:
results

LogisticRegression      LGBM
cv             0.779587  0.771627
test           0.777309  0.773791

На тестовой выборке вновь обе модели смогли превысить необходимый порог f1-меры, равный 0.75. Отличие в оценке моделей практически минимальное, LogisticRegression: 0.7795, LGBMClassifier: 0.7716.

Лучшая модель: LGBMClassifier.

## Выводы

В данном проекте мы построили модель классификации текстов, определяющую токсичный комментарий или нет. Для построения модели нам был предоставлен датасет, содержащий 2 признака:
* text - комментарий от пользователя на английском языке, самый длинный комментарий 5000 символов, самый короткий - 5 символов.
* target - целевая переменная, метка 0 или 1, характеризующая комментарий как токсичный

В самом датасете 159292 строки, нет пропусков, нет дубликатов. Сами комментарии представляют из себя строку, состоящую из предложений или законченных фраз, встречаются так же лишние символы в строках, сокращения, даты, цифры.

Мы лемматизировали все комментарии в датасете, очистили их от лишних символов, и обучили модели, используя Tfidf трансформацию признаков. Для обучения моделей и их улучшения мы использовали gridsearchcv, вместе с кросс-валидацией. Обе модели на кросс-валидационных выборках смогли превысить требуемое значение качества, при этом модель логистической регрессии имела более высокую оценку (0.7827), чем модель градиентного бустинга (0.77143).

На тестовой выборке вновь обе модели смогли превысить необходимый порог f1-меры, равный 0.75. Отличие в оценке моделей практически минимальное, LogisticRegression: 0.7795, LGBMClassifier: 0.7716.

Лучшая модель: LogisticRegression.  
Ее параметры:

In [41]:
print(grid_LGBM.best_estimator_)

Pipeline(steps=[('tfidf',
                 TfidfVectorizer(stop_words={'a', 'about', 'above', 'after',
                                             'again', 'against', 'ain', 'all',
                                             'am', 'an', 'and', 'any', 'are',
                                             'aren', "aren't", 'as', 'at', 'be',
                                             'because', 'been', 'before',
                                             'being', 'below', 'between',
                                             'both', 'but', 'by', 'can',
                                             'couldn', "couldn't", ...})),
                ('clf', LGBMClassifier(n_estimators=500, random_state=42))])
